In [1]:
import numpy as np

In [2]:
#menentukan bentuk lingkungan (yaitu, statusnya)
environment_rows = 11
environment_columns = 11

#buat array numpy 3D untuk menampung nilai Q saat ini untuk setiap pasangan status dan tindakan: Q(s, a)
q_values = np.zeros((environment_rows, environment_columns, 4))

In [3]:
#definisikan tindakan
#kode tindakan numerik: 0 = up, 1 = right, 2 = down, 3 = left
actions = ['up', 'right', 'down', 'left']

In [4]:
#Buat array numpy 2D untuk menampung hadiah untuk setiap negara bagian.
#Array berisi 11 baris dan 11 kolom (agar sesuai dengan bentuk lingkungan), dan setiap nilai diinisialisasi ke -100.
rewards = np.full((environment_rows, environment_columns), -100.)
rewards[0, 5] = 100. #atur hadiah untuk area pengemasan (yaitu sasaran) menjadi 100

#tentukan lokasi lorong (yaitu kotak putih) untuk baris 1 hingga 9
aisles = {} #menyimpan lokasi di kamus
aisles[1] = [i for i in range(1, 10)]
aisles[2] = [1, 7, 9]
aisles[3] = [i for i in range(1, 8)]
aisles[3].append(9)
aisles[4] = [3, 7]
aisles[5] = [i for i in range(11)]
aisles[6] = [5]
aisles[7] = [i for i in range(1, 10)]
aisles[8] = [3, 7]
aisles[9] = [i for i in range(11)]

#atur hadiah untuk semua lokasi lorong (yaitu kotak putih)
for row_index in range(1, 10):
  for column_index in aisles[row_index]:
    rewards[row_index, column_index] = -1.
  
#print rewards matrix
for row in rewards:
  print(row)

[-100. -100. -100. -100. -100.  100. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100.   -1. -100. -100. -100. -100. -100.   -1. -100.   -1. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100.   -1. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.]


In [5]:
#mendefinisikan fungsi yang menentukan apakah lokasi yang ditentukan adalah status terminal
def is_terminal_state(current_row_index, current_column_index):
  #jika hadiah untuk lokasi ini adalah -1, maka lokasi tersebut bukan status terminal (yakni 'kotak putih')
  if rewards[current_row_index, current_column_index] == -1.:
    return False
  else:
    return True

In [6]:
#mendefinisikan fungsi yang akan memilih lokasi awal non-terminal secara acak
def get_starting_location():
  #dapatkan indeks baris dan kolom acak
  current_row_index = np.random.randint(environment_rows)
  current_column_index = np.random.randint(environment_columns)
  #lanjutkan memilih indeks baris dan kolom acak hingga status non-terminal teridentifikasi
  #(yaitu, hingga negara bagian yang dipilih adalah 'kotak putih').
  while is_terminal_state(current_row_index, current_column_index):
    current_row_index = np.random.randint(environment_rows)
    current_column_index = np.random.randint(environment_columns)
  return current_row_index, current_column_index

In [7]:
#tentukan algoritma serakah epsilon yang akan memilih tindakan mana yang harus diambil selanjutnya (yaitu, ke mana harus bergerak selanjutnya)
def get_next_action(current_row_index, current_column_index, epsilon):
  #jika nilai yang dipilih secara acak antara 0 dan 1 lebih kecil dari epsilon, 
  #lalu pilih nilai yang paling menjanjikan dari tabel Q untuk keadaan ini.
  if np.random.random() < epsilon:
    return np.argmax(q_values[current_row_index, current_column_index])
  else: #memilih tindakan acak
    return np.random.randint(4)

In [8]:
#tentukan fungsi yang akan mendapatkan lokasi berikutnya berdasarkan tindakan yang dipilih
def get_next_location(current_row_index, current_column_index, action_index):
  new_row_index = current_row_index
  new_column_index = current_column_index
  if actions[action_index] == 'up' and current_row_index > 0:
    new_row_index -= 1
  elif actions[action_index] == 'right' and current_column_index < environment_columns - 1:
    new_column_index += 1
  elif actions[action_index] == 'down' and current_row_index < environment_rows - 1:
    new_row_index += 1
  elif actions[action_index] == 'left' and current_column_index > 0:
    new_column_index -= 1
  return new_row_index, new_column_index

In [9]:
#Tentukan fungsi yang akan mendapatkan jalur terpendek antara lokasi mana pun di dalam gudang itu
#robot diperbolehkan melakukan perjalanan dan lokasi pengemasan barang.
def get_shortest_path(start_row_index, start_column_index):
  #segera kembali jika ini adalah lokasi awal yang tidak valid
  if is_terminal_state(start_row_index, start_column_index):
    return []
  else: #jika ini adalah lokasi awal yang 'legal'
    current_row_index, current_column_index = start_row_index, start_column_index
    shortest_path = []
    shortest_path.append([current_row_index, current_column_index])
    #terus bergerak sepanjang jalan sampai kita mencapai tujuan (yaitu lokasi pengemasan barang)
    while not is_terminal_state(current_row_index, current_column_index):
      #mendapatkan tindakan terbaik untuk diambil
      action_index = get_next_action(current_row_index, current_column_index, 1.)
      #pindah ke lokasi berikutnya di jalur tersebut, dan tambahkan lokasi baru ke daftar
      current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
      shortest_path.append([current_row_index, current_column_index])
    return shortest_path

In [10]:
#menentukan parameter pelatihan
epsilon = 0.9 #persentase waktu ketika kita harus mengambil tindakan terbaik (bukan tindakan acak)
discount_factor = 0.9 #faktor diskon untuk imbalan di masa depan
learning_rate = 0.9 #tingkat di mana agen AI harus belajar

#dijalankan melalui 1000 episode pelatihan
for episode in range(1000):
  #dapatkan lokasi awal untuk episode ini
  row_index, column_index = get_starting_location()

  #terus mengambil tindakan (yaitu bergerak) sampai kita mencapai keadaan terminal
  #(yaitu, sampai kita mencapai area pengemasan barang atau menabrak lokasi penyimpanan barang)
  while not is_terminal_state(row_index, column_index):
    #memilih tindakan mana yang akan diambil (yaitu, ke mana harus bergerak selanjutnya)
    action_index = get_next_action(row_index, column_index, epsilon)

    #melakukan tindakan yang dipilih, dan beralih ke keadaan berikutnya (yaitu, pindah ke lokasi berikutnya)
    old_row_index, old_column_index = row_index, column_index
    row_index, column_index = get_next_location(row_index, column_index, action_index)
    
    #menerima hadiah untuk pindah ke negara bagian baru, dan menghitung perbedaan temporal
    reward = rewards[row_index, column_index]
    old_q_value = q_values[old_row_index, old_column_index, action_index]
    temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

    #perbarui nilai Q untuk pasangan status dan tindakan sebelumnya
    new_q_value = old_q_value + (learning_rate * temporal_difference)
    q_values[old_row_index, old_column_index, action_index] = new_q_value

print('Training complete!')

Training complete!


In [11]:
#menampilkan beberapa jalur terpendek
print(get_shortest_path(3, 9)) #starting at row 3, column 9
print(get_shortest_path(5, 0)) #starting at row 5, column 0
print(get_shortest_path(9, 5)) #starting at row 9, column 5

[[3, 9], [2, 9], [1, 9], [1, 8], [1, 7], [1, 6], [1, 5], [0, 5]]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6], [5, 7], [4, 7], [3, 7], [2, 7], [1, 7], [1, 6], [1, 5], [0, 5]]
[[9, 5], [9, 6], [9, 7], [8, 7], [7, 7], [7, 6], [7, 5], [6, 5], [5, 5], [5, 6], [5, 7], [4, 7], [3, 7], [2, 7], [1, 7], [1, 6], [1, 5], [0, 5]]


In [12]:
#tampilkan contoh jalur terpendek terbalik
path = get_shortest_path(5, 2) #go to row 5, column 2
path.reverse()
print(path)

[[0, 5], [1, 5], [1, 6], [1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [5, 6], [5, 5], [5, 4], [5, 3], [5, 2]]
